In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import genius_cred as gc
import spotify_cred as sc
import spotify_api_client 
from spotify_api_client import SpotifyAPI 
from difflib import SequenceMatcher
import base64
from urllib.parse import urlencode
import datetime as dt
import math

import warnings
warnings.filterwarnings('ignore')

# Scrape Billboard Top 100

In [23]:
base_url = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs/'
years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [144]:
### 2011 and 2016 have 99 songs instead of 100 

### Scrape top 100 songs by year
def get_songs(year):
    base_url = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs/'
    
    song_list = []
    page = requests.get(base_url.format(year))
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all(id='title-of-a-story')
    
    if year in (2011, 2016):
        for i in results[:99]:
            song_name = i.text.replace('\n', '').replace('\t','')
            song_list.append(song_name)
    else:
        for i in results[:100]:
            song_name = i.text.replace('\n', '').replace('\t','')
            song_list.append(song_name)
        
    return song_list

### Scrape artist names for songs
def get_artists(year):
    base_url = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs/'
    
    page = requests.get(base_url.format(year))
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all('span', class_='c-label')
    
    artist_list = [results[i].text.replace('\n', '').replace('\t','') for i in range(1, len(results)+1, 2)]
    
    return artist_list

In [145]:
### Aggregating data across all years
year_col= []
song_col = []
artist_col = []# 2011, 2016 missing 1 value each

for year in years: 
    if year in (2011, 2016):
        year_100 = np.zeros(99)+year
    else:
        year_100 = np.zeros(100)+year
    year_col = np.concatenate((year_col, year_100), axis=0) 
    
    songs = get_songs(year)
    song_col = np.concatenate((song_col, songs), axis=0)
    
    artists = get_artists(year)
    artist_col = np.concatenate((artist_col, artists), axis=0)

In [147]:
# Creating df
charts = dict()
charts['year'] = year_col
charts['song'] = song_col
charts['artist'] = artist_col 

charts_df = pd.DataFrame(charts)

In [148]:
charts_df.head()

,year,song,artist
0,2011.0,Rolling In The Deep,Adele
1,2011.0,Party Rock Anthem,LMFAO Featuring Lauren Bennett & GoonRock
2,2011.0,Firework,Katy Perry
3,2011.0,E.T.,Katy Perry Featuring Kanye West
4,2011.0,Give Me Everything,"Pitbull Featuring Ne-Yo, Afrojack & Nayer"


# Collect lyrics from Genius API

In [2]:
genius_url = 'https://api.genius.com/'
client_access_token = gc.token

In [3]:
r = requests.get(genius_url+f'search?access_token={client_access_token}')
r.status_code

200

In [4]:
def similarity(string1, string2):
    sim = SequenceMatcher(None, string1, string2).ratio()
    
    return sim

def get_url(song, artist):
    song = re.sub(r'[^\w\s]', '', song)
    song = song.replace(' ', '-')
    genius_search_url = f"http://api.genius.com/search?q={song}&access_token={client_access_token}"
    response = requests.get(genius_search_url)
    json = response.json()
    
    url = None
    for hit in json['response']['hits']:
        if similarity(artist, hit['result']['primary_artist']['name']) > 0.5 or artist[:5] in hit['result']['primary_artist']['name']:
            url = hit['result']['url']
            break
    if url:
        pass    
    
    return url

def get_lyrics(url):
    lyrics = None
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    html = soup.find('div', class_="Lyrics__Container-sc-1ynbvzw-6 YYrds")
    lyrics = html.text
    
    return lyrics

In [64]:
### Get urls for lyrics pages
lyric_urls = []
for song, artist in zip(charts_df.song, charts_df.artist):
    lyric_url = get_url(song, artist)
    lyric_urls.append(lyric_url)
    counter += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [77]:
charts_df = pd.read_csv('charts.csv')
charts_df.head()

,Unnamed: 0,year,song,artist,lyric_url
0,0,2011.0,Rolling In The Deep,Adele,https://genius.com/Adele-rolling-in-the-deep-l...
1,1,2011.0,Party Rock Anthem,LMFAO Featuring Lauren Bennett & GoonRock,https://genius.com/Lmfao-party-rock-anthem-lyrics
2,2,2011.0,Firework,Katy Perry,https://genius.com/Katy-perry-firework-lyrics
3,3,2011.0,E.T.,Katy Perry Featuring Kanye West,https://genius.com/Katy-perry-et-remix-lyrics
4,4,2011.0,Give Me Everything,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",https://genius.com/Pitbull-give-me-everything-...


In [41]:
### From Genius URLs, get lyrics for wach song 
song_lyrics = []
counter = 0
for url in charts_df.lyric_url:
    if type(url) != type('string'): 
        continue
    else:
        lyric = get_lyrics(url)
        song_lyrics.append(lyric)
    counter += 1
    print(counter)

1
2


KeyboardInterrupt: 

In [46]:
### Batching lyrics URLs because request times out
begin = np.linspace(0, 975, 40)
end = list(np.linspace(25,975, 39)) + [len(charts_df)-1]
song_lyrics = []
counter = 0

for i, j in zip(begin, end):
    i = int(i)
    j = int(j)
    for url in charts_df.lyric_url[i:j]:
        if type(url) != str:
            continue
        else:
            lyric= get_lyrics(url)
            song_lyrics.append(lyric)
        counter += 1
        print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [49]:
saved_lyrics = song_lyrics

In [60]:
len(song_lyrics)

997

In [50]:
for url in charts_df.lyric_url[924:]:
    if type(url) != str:
        continue
    else: 
        lyric = get_lyrics(url)
        saved_lyrics.append(lyric)
        counter += 1
        print(counter)

927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997


In [75]:
for i in range(len(charts_df)):
    truth = get_lyrics(charts_df.lyric_url[i]) in saved_lyrics
    print(i,truth)

0 True
1 True
2 False
3 True
4 False
5 True
6 True


KeyboardInterrupt: 

In [58]:
charts_pop_df = pd.read_csv('charts_pop.csv')
charts_pop_df= charts_pop_df.iloc[:,1:]
charts_pop_df.head()

,year,song,artist,lyric_url,popularity
0,2011.0,Rolling In The Deep,Adele,https://genius.com/Adele-rolling-in-the-deep-l...,79.0
1,2011.0,Party Rock Anthem,LMFAO Featuring Lauren Bennett & GoonRock,https://genius.com/Lmfao-party-rock-anthem-lyrics,0.0
2,2011.0,Firework,Katy Perry,https://genius.com/Katy-perry-firework-lyrics,0.0
3,2011.0,E.T.,Katy Perry Featuring Kanye West,https://genius.com/Katy-perry-et-remix-lyrics,47.0
4,2011.0,Give Me Everything,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",https://genius.com/Pitbull-give-me-everything-...,23.0


In [59]:
lyrics_df = pd.DataFrame()
lyrics_df['song'] = charts_pop_df.song
lyrics_df['artist'] = charts_pop_df.artist
lyrics_df['lyrics'] = saved_lyrics

ValueError: Length of values (997) does not match length of index (998)

In [272]:
no_urls = charts_df[charts_df.lyric_url.isna()]
no_urls

,year,song,artist,lyric_url
17,2011.0,F**kin' Perfect,P!nk,None
31,2011.0,Blow,Ke$ha,None
59,2011.0,More,Usher,None
71,2011.0,Without You,David Guetta Featuring Usher,None
80,2011.0,Remind Me,Brad Paisley Duet With Carrie Underwood,None
...,...,...,...,...
825,2019.0,Ran$om,Lil Tecca,None
840,2019.0,ME!,Taylor Swift Featuring Brendon Urie,None
920,2020.0,"10,000 Hours",Dan + Shay & Justin Bieber,None
969,2020.0,Hot,Young Thug Featuring Gunna,None


In [270]:
np.unique(no_urls.song)

array(['#Beautiful', '1-800-273-8255', '10,000 Hours', 'Animal',
       'Animals', 'B**** Better Have My Money', 'Backseat', 'Bad',
       'Big Bank', 'Blow', 'Bodak Yellow (Money Moves)', 'Brave',
       'Broccoli', 'Burn', 'Chains', 'Cheap Thrills', 'Clarity', 'Close',
       'Cold', 'Come Over', 'Dirt', 'Down', 'Downtown', 'F**kin Problems',
       "F**kin' Perfect", 'Ghost', 'Good Girl', 'H.O.L.Y.', 'Heaven',
       'Holy Grail', 'Home', 'Hot', 'Hot Boy', 'Hurricane',
       "I Don't F**k With You",
       "I Don't Wanna Live Forever (Fifty Shades Darker)", 'In The Dark',
       'Just Give Me A Reason', "Let's Go", 'Lights', 'Lights Down Low',
       'Love Galore', 'ME!', 'Me And My Broken Heart', 'More', 'My Hitta',
       'Ni**as in Paris', 'No', 'No Mediocre', 'P*$$y Fairy (OTW)',
       'Pillowtalk', 'Ran$om', 'Remind Me', 'Springsteen',
       'Stay The Night', 'Summer',
       'Sunflower (Spider-Man: Into The Spider-Verse)', 'Taste',
       'Tuesday', 'Wanted', 'We Are Young'

Issues:
- When punctuation is part of title 
- Censored titles 
- Additional descriptions in title

In [276]:
re.sub(r'[^\w\s]','', "F**kin' Perfect" )

'Fkin Perfect'

In [381]:
charts_df.to_csv('charts.csv')


# Collect song metrics from Spotify API 

### Get popularity metrics

In [28]:
charts_df = pd.read_csv('charts.csv').iloc[:,1:]

In [25]:
spotify_client_id = sc.client_id 
spotify_secret = sc.client_secret
spotify = SpotifyAPI(spotify_client_id, spotify_secret)

In [27]:
spotify.perform_auth()

True

In [33]:
popularity_col = []
for song, artist in zip(charts_df.song, charts_df.artist):
    pop = spotify.get_popularity(song, artist)
    popularity_col.append(pop)

In [35]:
charts_df['popularity'] = popularity_col
charts_df.to_csv('charts_pop.csv')

### Get audio features

In [36]:
charts_df.head()

,year,song,artist,lyric_url,popularity
0,2011.0,Rolling In The Deep,Adele,https://genius.com/Adele-rolling-in-the-deep-l...,79.0
1,2011.0,Party Rock Anthem,LMFAO Featuring Lauren Bennett & GoonRock,https://genius.com/Lmfao-party-rock-anthem-lyrics,0.0
2,2011.0,Firework,Katy Perry,https://genius.com/Katy-perry-firework-lyrics,0.0
3,2011.0,E.T.,Katy Perry Featuring Kanye West,https://genius.com/Katy-perry-et-remix-lyrics,47.0
4,2011.0,Give Me Everything,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",https://genius.com/Pitbull-give-me-everything-...,23.0


In [45]:
audio_features_df = pd.DataFrame()

for song, artist in zip(charts_df.song, charts_df.artist):
    feature_dict = spotify.get_audio_features(song, artist)
    audio_features_df = audio_features_df.append(feature_dict, ignore_index=True)

In [51]:
audio_features_df['song'] = charts_df.song
audio_features_df['artist'] = charts_df.artist

In [53]:
audio_features_df = audio_features_df[['song', 'artist', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature']]

audio_features_df.to_csv('audio_features.csv')

In [57]:
audio_features_df.isna().sum()

song                 0
artist               0
danceability        28
energy              28
key                 28
loudness            28
mode                28
speechiness         28
acousticness        28
instrumentalness    28
liveness            28
valence             28
tempo               28
type                28
id                  28
uri                 28
track_href          28
analysis_url        28
duration_ms         28
time_signature      28
dtype: int64